# Time Series Forecasting - Training Pipeline

This notebook trains forecasting models for anonymized time series data.

## Overview
- Load and preprocess time series data
- Create features (lags, rolling stats, technical indicators)
- Train multiple models (XGBoost, CatBoost, LightGBM, LSTM, Transformer)
- Evaluate models using R², MAE, RMSE
- Create ensemble for robust predictions


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import pipeline
from pipeline import run_pipeline, TimeSeriesPipeline
from config import *
from utils.evaluation import print_metrics
from utils.visualization import plot_forecast, plot_residuals, plot_model_comparison


## 1. Load Data


In [ ]:
# Load training data
# Replace with your actual data path
data_path = DATA_DIR / "train.csv"

# If data is not in expected format, specify column names
# series_columns = ['series_1', 'series_2', 'series_3']  # Adjust as needed

# Initialize pipeline
pipeline = TimeSeriesPipeline()

# Load data
series_data = pipeline.load_and_prepare_data(data_path)

# Display summary
for name, df in series_data.items():
    print(f"\n{name}:")
    print(f"  Shape: {df.shape}")
    print(f"  Date range: {df.index.min()} to {df.index.max()}")
    print(f"  Mean: {df.iloc[:, 0].mean():.2f}")
    print(f"  Std: {df.iloc[:, 0].std():.2f}")


## 2. Exploratory Data Analysis


In [ ]:
# Visualize each series
fig, axes = plt.subplots(len(series_data), 1, figsize=(14, 5*len(series_data)))
if len(series_data) == 1:
    axes = [axes]

for idx, (name, df) in enumerate(series_data.items()):
    ax = axes[idx]
    ax.plot(df.index, df.iloc[:, 0], linewidth=1.5)
    ax.set_title(f"Time Series: {name}", fontsize=14, fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Value')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / "time_series_overview.png", dpi=300, bbox_inches='tight')
plt.show()


## 3. Train Models


In [ ]:
# Train models for each series
all_results = {}

for series_name, df in series_data.items():
    print(f"\n{'='*60}")
    print(f"Training models for: {series_name}")
    print(f"{'='*60}")
    
    # Prepare features
    target_col = df.columns[0]
    X, y = pipeline.prepare_features(series_name, df, target_col)
    
    # Split train/validation
    split_idx = int(len(X) * (1 - pipeline.config['validation_split']))
    X_train, X_val = X[:split_idx], X[split_idx:]
    y_train, y_val = y[:split_idx], y[split_idx:]
    
    print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}")
    
    # Train models
    pipeline.train_models(series_name, X_train, y_train, X_val, y_val)
    
    # Evaluate on validation set
    results = pipeline.evaluate_all_models(series_name, X_val, y_val)
    all_results[series_name] = results
    
    # Print metrics
    print_metrics(results.get('ensemble', {}), f"{series_name} - Ensemble Model")


## 4. Model Comparison


In [ ]:
# Compare models across all series
for series_name, results in all_results.items():
    # Extract R² scores
    model_comparison = {}
    for model_name, metrics in results.items():
        r2_key = f"{series_name}_{model_name}_r2_score"
        model_comparison[model_name] = {r2_key: metrics.get(r2_key, 0)}
    
    plot_model_comparison(
        {k: v for d in model_comparison.values() for k, v in d.items()},
        metric=list(list(model_comparison.values())[0].keys())[0],
        save_path=FIGURES_DIR / f"{series_name}_model_comparison.png"
    )


## 5. Visualize Predictions


In [ ]:
# Generate predictions and visualize for each series
for series_name, df in series_data.items():
    # Prepare features
    target_col = df.columns[0]
    X, y = pipeline.prepare_features(series_name, df, target_col)
    
    # Split
    split_idx = int(len(X) * (1 - pipeline.config['validation_split']))
    X_train, X_val = X[:split_idx], X[split_idx:]
    y_train, y_val = y[:split_idx], y[split_idx:]
    
    # Get predictions from ensemble
    y_pred = pipeline.predict(series_name, X_val, model_name='ensemble')
    
    # Get corresponding dates
    dates_train = df.index[:split_idx + pipeline.config['lookback_window']]
    dates_val = df.index[split_idx + pipeline.config['lookback_window']:]
    
    # Convert to series for plotting
    y_train_series = pd.Series(y_train, index=dates_train[-len(y_train):])
    y_val_series = pd.Series(y_val, index=dates_val[:len(y_val)])
    y_pred_series = pd.Series(y_pred, index=dates_val[:len(y_pred)])
    
    # Plot forecast
    plot_forecast(
        y_train_series, y_val_series, y_pred_series,
        title=f"Forecast: {series_name}",
        save_path=FIGURES_DIR / f"{series_name}_forecast.png"
    )
    
    # Plot residuals
    plot_residuals(
        y_val_series, y_pred_series,
        title=f"Residuals Analysis: {series_name}",
        save_path=FIGURES_DIR / f"{series_name}_residuals.png"
    )


## 6. Save Models


In [ ]:
# Save trained models
pipeline.save_models()
print("\nModels saved successfully!")


## 7. Summary


In [ ]:
# Print final summary
print("\n" + "="*60)
print("FINAL RESULTS SUMMARY")
print("="*60)

for series_name, results in all_results.items():
    print(f"\n{series_name}:")
    for model_name, metrics in results.items():
        r2_key = f"{series_name}_{model_name}_r2_score"
        mae_key = f"{series_name}_{model_name}_mae"
        rmse_key = f"{series_name}_{model_name}_rmse"
        
        r2 = metrics.get(r2_key, 0)
        mae = metrics.get(mae_key, 0)
        rmse = metrics.get(rmse_key, 0)
        
        print(f"  {model_name:20s}: R²={r2:7.4f}, MAE={mae:8.2f}, RMSE={rmse:8.2f}")
